In [1]:
import os
import openai
from llama_index import SimpleDirectoryReader
from llama_index.embeddings import HuggingFaceEmbedding

from dotenv import load_dotenv

In [2]:
# loading the openai key
load_dotenv()
open_api_key=os.environ['OPENAI_API_KEY']
openai.api_key=open_api_key

In [3]:
# Loading and cchunking the documents

documents=SimpleDirectoryReader(
    input_files=["../data/Machine Learning Engineering with Python-2023.pdf"]
).load_data()


In [4]:
#Checking the summary of chunked documents
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[2].text)
print(len(documents[2].text))
print(len(documents[10].text))
print(len(documents[22].text))




<class 'list'> 

463 

<class 'llama_index.schema.Document'>
Machine Learning Engineering with Python
Second Edition
Copyright © 2023 Packt Publishing
All rights reserved. No part of this book may be reproduced, stored in a retrieval system, or transmitted in 
any form or by any means, without the prior written permission of the publisher, except in the case of brief 
quotations embedded in critical articles or reviews.
Every effort has been made in the preparation of this book to ensure the accuracy of the information 
presented. However, the information contained in this book is sold without warranty, either express or 
implied. Neither the author nor Packt Publishing or its dealers and distributors, will be held liable for any 
damages caused or alleged to have been caused directly or indirectly by this book.
Packt Publishing has endeavored to provide trademark information about all of the companies and products 
mentioned in this book by the appropriate use of capitals. However, Pa

## Basic RAG pipeline

In [5]:
# Combining the all chunked text into single documents
from llama_index import Document

documnet=Document(tetx="\n\n".join([doc.text for doc in documents]))
documnet

Document(id_='69b12ed1-34a5-487a-bc47-d87e5c515cc6', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='44136fa355b3678a1146ad16f7e8649e94fb4fc21fe77e8310c060f61caaff8a', text='', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')

In [6]:
# Creating index
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

#config llm
llm= OpenAI(model="gpt-3.5-turbo",temperature=0.1) 
#config embedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

service_context=ServiceContext.from_defaults(
    llm=llm,
    embed_model= embed_model
)

index= VectorStoreIndex.from_documents([documnet],
                                       service_context=service_context)

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

c:\Users\sarthakkaushik\.conda\envs\Adv_RAG_lamaindx_venv\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sarthakkaushik\AppData\Local\llama_index. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [7]:
query_engine= index.as_query_engine()

In [9]:
response=query_engine.query("How can we analyzing your own code for security issues")
print(str(response))

One way to analyze your own code for security issues is to conduct a thorough code review. This involves carefully examining your code line by line to identify any potential vulnerabilities or weaknesses. Additionally, you can use automated code analysis tools that can scan your code for common security issues and provide recommendations for improvement. Regularly testing your code in different scenarios and environments can also help uncover any security flaws.


## Evaluation setup using TruLens

In [13]:
eval_questions = []
with open('../eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

How can we analyzing your own code for security issues


In [14]:
# Adding new question:
new_question = "How to publish in ECR"
eval_questions.append(new_question)


In [15]:
print(eval_questions)

['How can we analyzing your own code for security issues', 'How to publish in ECR']


In [16]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [18]:
import sys
sys.path.append("..")
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")


✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [19]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

In [20]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [21]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Context Relevance,Answer Relevance,Groundedness,Context Relevance_calls,Answer Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_cb7363c0389ce919b5cd1da786d268e2,"""How can we analyzing your own code for securi...","""One way to analyze your own code for security...",-,"{""record_id"": ""record_hash_cb7363c0389ce919b5c...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-02T23:34:53.356450"", ""...",2023-12-02T23:34:58.570815,0.0,0.9,1.0,[{'args': {'question': 'How can we analyzing y...,[{'args': {'prompt': 'How can we analyzing you...,"[{'args': {'source': '', 'statement': 'One way...",5,202,0.000344
1,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_7e475eb8b2506afec9e27a2377f997a5,"""How to publish in ECR""","""To publish in ECR, you can follow these steps...",-,"{""record_id"": ""record_hash_7e475eb8b2506afec9e...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-02T23:34:58.822650"", ""...",2023-12-02T23:35:12.484323,0.0,1.0,NaN,[{'args': {'question': 'How to publish in ECR'...,"[{'args': {'prompt': 'How to publish in ECR', ...",NaN,13,302,0.000545
